# Sandbox
For playing around during development.

In [1]:
%pylab inline
import pysd
#print pysd.__version__
print pysd.__file__
import pandas as pd
import os




Populating the interactive namespace from numpy and matplotlib
pysd/__init__.pyc


/Users/houghton/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
pysd.read_vensim('./tests/test-models/tests/subscript_3d_arrays/test_subscript_3d_arrays.mdl')

AttributeError: 'module' object has no attribute '_stock_a_init'

In [ ]:
%debug

> /Users/houghton/Google_Drive/Academic Projects/PYSD/pysd/pysd/pysd.py(254)initialize_state()
    252                     initialization_order.append(key)
    253                 except KeyError:  # may also need to catch TypeError?
--> 254                     retry_flag = True
    255             if not making_progress:
    256                 raise KeyError('Unresolvable Reference: Probable circular initialization'+

ipdb> model.components
*** NameError: name 'model' is not defined
ipdb> ls
*** NameError: name 'ls' is not defined


In [47]:
np.ones(3)*NaN

array([ nan,  nan,  nan])

In [2]:
import xarray as xr

In [4]:
coords = {'Second Dimension Subscript': ['Column 1', 'Column 2'],
          'Third Dimension Subscript': ['Depth 1', 'Depth 2'],
          'One Dimensional Subscript': ['Entry 1', 'Entry 2', 'Entry 3']}
coords

{'One Dimensional Subscript': ['Entry 1', 'Entry 2', 'Entry 3'],
 'Second Dimension Subscript': ['Column 1', 'Column 2'],
 'Third Dimension Subscript': ['Depth 1', 'Depth 2']}

In [43]:
a = xr.DataArray(data=np.zeros(map(len, coords.values()))*NaN, coords=coords)

In [72]:
m = np.empty([3])*NaN
m

array([ nan,  nan,  nan])

In [70]:
np.empty(4)

array([ 1.,  2.,  3.,  4.])

In [49]:
a.sel(One Dimensional Subscript='Column 1')

SyntaxError: invalid syntax (<ipython-input-49-f3f8947e1e9d>, line 1)

In [36]:
a.loc[{'One Dimensional Subscript': ['Entry 1']}]

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2, One Dimensional Subscript: 1)>
array([[[ 1.],
        [ 2.]],

       [[ 3.],
        [ 4.]]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
  * One Dimensional Subscript   (One Dimensional Subscript) |S7 'Entry 1'

In [ ]:
a.loc[{'One Dimensional Subscript': ['Entry 1']}]

In [18]:
a.loc[:,:,'Entry 1']

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2)>
array([[ 1.,  2.],
       [ 0.,  0.]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
    One Dimensional Subscript   |S7 'Entry 1'

In [37]:
a.loc[:,'Entry 1',:]

KeyError: 'Entry 1'

In [16]:
a.loc['Column 1',:,'Entry 1']

<xarray.DataArray (Third Dimension Subscript: 2)>
array([ 0.,  0.])
Coordinates:
    Second Dimension Subscript  |S8 'Column 1'
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
    One Dimensional Subscript   |S7 'Entry 1'

In [17]:
a.loc['Column 1',:,'Entry 1'] = 1, 2

In [21]:
a.loc[:,:,'Entry 1'] = [1,2],[3,4]

In [22]:
a

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2, One Dimensional Subscript: 3)>
array([[[ 1.,  0.,  0.],
        [ 2.,  0.,  0.]],

       [[ 3.,  0.,  0.],
        [ 4.,  0.,  0.]]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
  * One Dimensional Subscript   (One Dimensional Subscript) |S7 'Entry 1' ...

In [25]:
a.loc[:,:,'Entry 1'] = [1,2],[3,4]

In [26]:
a

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2, One Dimensional Subscript: 3)>
array([[[ 1.,  0.,  0.],
        [ 2.,  0.,  0.]],

       [[ 3.,  0.,  0.],
        [ 4.,  0.,  0.]]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
  * One Dimensional Subscript   (One Dimensional Subscript) |S7 'Entry 1' ...

In [50]:
m = {'Dim1': ['A', 'B'],
                                      'Dim2': ['C', 'D', 'E'],
                                      'Dim3': ['F', 'G', 'H', 'I']}

In [53]:
m.iterkeys()

<dictionary-keyiterator at 0x10d7459f0>

In [56]:
a = 'hi'
a = a or None
a

'hi'

In [57]:
b = None
b = b or 'hi'
b

'hi'

In [60]:
c = 'hi there'
c = c or b
c

'hi there'

In [61]:
b = None
b = b + 'hi'
b

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [62]:
a = ['hi']
a += ['bob']
a

['hi', 'bob']

In [66]:
 or 'hi'

' '

In [64]:
None + 'hi'  'there'

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [73]:
import autopep8

In [82]:
print autopep8.fix_code("xr.DataArray(data=np.empty([3])*NaN, coords={'Dim1': ['A','B','C'], 'Dim1': ['A', 'B', 'C'], 'Dim1': ['A', 'B', 'C'] 'Dim2': ['D', 'E', 'F']})')", options={'aggressive': 1})

xr.DataArray(data=np.empty([3]) * NaN, coords={'Dim1': ['A', 'B', 'C'], 'Dim1': ['A', 'B', 'C'], 'Dim1': ['A', 'B', 'C'] 'Dim2': ['D', 'E', 'F']})')



In [79]:
len("xr.DataArray(data=np.empty([3])*NaN, coords={'Dim1': ['A', 'B', 'C'], 'Dim1': ['A', 'B', 'C'], 'Dim1': ['A', 'B', 'C'] 'Dim2': ['D', 'E', 'F']})'))")

147

In [84]:
repr(map(len, coords.values()))

'[2, 2, 3]'

In [85]:
import textwrap

In [ ]:
textwrap.

In [86]:
string = """hi
this
is 
my
string"""


In [ ]:
xr.